# Few-Shot Prompting

In some cases, it's easier to show the model what you want rather than tell the model what you want.
<br><br>
One way to show the model what you want is with creating a few fake back-and-forth messages between user and assistant. This is called `few-shot prompting`. 
<br><br>
The opposite of few-shot prompting is `zero-shot prompting` (previous examples).
***

## Prerequisites

1. Make sure that `python3` is installed on your system.
1. Create and Activate a Virtual Environment: <br><br>
    `python3 -m venv venv` <br>
    `source venv/bin/activate` <br><br>
1. Create a `.env` file in the same directory as this script and add the following variables:<br><br>
     ```
     AZURE_OPENAI_ENDPOINT=<your_azure_openai_endpoint>
     AZURE_OPENAI_MODEL=<your_azure_openai_model>
     AZURE_OPENAI_API_VERSION=<your_azure_openai_api_version>
     AZURE_OPENAI_API_KEY=<your_azure_openai_api_key>
     ```
***

## Install Dependencies

The required libraries are listed in the requirements.txt file. Use the following command to install them:

In [11]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


***
## Import Modules

In [12]:
from openai import AzureOpenAI  # The `AzureOpenAI` library is used to interact with the Azure OpenAI API.
from dotenv import load_dotenv  # The `dotenv` library is used to load environment variables from a .env file.
import os                       # Used to get the values from environment variables.
from pprint import pprint       # The `pprint` library is used to pretty-print a dictionary

## Load environment variables from .env file

The `load_dotenv()` function reads the .env file and loads the variables as env variables, making them accessible via `os.environ` or `os.getenv()`.

In [13]:
load_dotenv()

AZURE_OPENAI_ENDPOINT        = os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_OPENAI_MODEL           = os.environ['AZURE_OPENAI_MODEL']
AZURE_OPENAI_API_VERSION     = os.environ['AZURE_OPENAI_VERSION']
AZURE_OPENAI_API_KEY         = os.environ['AZURE_OPENAI_API_KEY']

## Create an instance of the AzureOpenAI client
- The `AzureOpenAI` class is part of the `openai` library, which is used to interact with the Azure OpenAI API.
- It requires the Azure endpoint, API key, and API version to be passed as parameters.

In [14]:
client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_key = AZURE_OPENAI_API_KEY,  
    api_version = AZURE_OPENAI_API_VERSION
)

## Set the behavior or personality of the assistant by providing fake conversations

In this example, we are expecting the assistant to respond in Hindi.
<br><br>
The `conversation` list contains a series of messages that simulate such conversation.
<br><br>
To help clarify that the example messages are not part of a real conversation, and shouldn't be referred back by the model, set the message role as `system` followed by a `name` field. The value of `name` field can either by `example_user` or `example_assistant`

In [15]:
conversation=[
        {"role": "system", "content": "You answer based on the pattern of the conversation."},
        {"role": "system", "name":"example_user", "content": "Hi, how are you?"},
        {"role": "system", "name": "example_assistant", "content": "Main accha hoon, aap kaise hain?"},
        {"role": "system", "name":"example_user", "content": "I am fine, can you tell me something?"},
        {"role": "system", "name": "example_assistant", "content": "Haan, bilkul! Aapko kya jaanana hai?"}
    ]

## Call the Azure OpenAI API to get the AI's response. Append the assistant's response to the conversation history

- Append the `conversation` array with user's question
- Call the Azure OpenAI API to get the AI's response
- Append the AI's response to the `conversation`

Rinse and repeat (put the logic in a function)

In [16]:
def talk_ai(question):
    
    # --------------------------------------------------------------
    # Append user question to the conversation history
    # --------------------------------------------------------------
    conversation.append({"role": "user", "content": question})

    try:
        # --------------------------------------------------------------
        # Send the conversation history to Azure OpenAI API to get the AI's response
        # --------------------------------------------------------------
        response = client.chat.completions.create(
            model= AZURE_OPENAI_MODEL, # model = "deployment_name".
            messages=conversation,
            temperature=0.7, # Control randomness (0 = deterministic, 1 = creative)
            max_tokens=1000  # Limit the length of the response
        )

        # --------------------------------------------------------------
        # Append the assistant's response to the conversation history
        # --------------------------------------------------------------
        conversation.append({"role": "assistant", "content": response.choices[0].message.content})
        
        # --------------------------------------------------------------
        # Debug: Print the entire conversation history
        # --------------------------------------------------------------
        print("\nDEBUG: Conversation history:\n")
        pprint(conversation)

        return response
    except Exception as e:
        print(f"Error getting answer from AI: {e}")

## Prompt user for question, get response from LLM

In [17]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)

print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)


Question: Hello friend

DEBUG: Conversation history:

[{'content': 'You answer based on the pattern of the conversation.',
  'role': 'system'},
 {'content': 'Hi, how are you?', 'name': 'example_user', 'role': 'system'},
 {'content': 'Main accha hoon, aap kaise hain?',
  'name': 'example_assistant',
  'role': 'system'},
 {'content': 'I am fine, can you tell me something?',
  'name': 'example_user',
  'role': 'system'},
 {'content': 'Haan, bilkul! Aapko kya jaanana hai?',
  'name': 'example_assistant',
  'role': 'system'},
 {'content': 'Hello friend', 'role': 'user'},
 {'content': 'Namaste dost', 'role': 'assistant'}]

Answer from AI:
Namaste dost


## Ask again

In [18]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)

print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)

Question: How to excel as a devOps engineer?

DEBUG: Conversation history:

[{'content': 'You answer based on the pattern of the conversation.',
  'role': 'system'},
 {'content': 'Hi, how are you?', 'name': 'example_user', 'role': 'system'},
 {'content': 'Main accha hoon, aap kaise hain?',
  'name': 'example_assistant',
  'role': 'system'},
 {'content': 'I am fine, can you tell me something?',
  'name': 'example_user',
  'role': 'system'},
 {'content': 'Haan, bilkul! Aapko kya jaanana hai?',
  'name': 'example_assistant',
  'role': 'system'},
 {'content': 'Hello friend', 'role': 'user'},
 {'content': 'Namaste dost', 'role': 'assistant'},
 {'content': 'How to excel as a devOps engineer?', 'role': 'user'},
 {'content': 'DevOps engineer banne ke liye, aapko coding aur scripting, Linux '
             'fundamentals, aur DevOps tools jaise ki Jenkins, Docker, aur '
             'Kubernetes par acchi samajh honi chahiye. Continuous '
             'Integration/Continuous Deployment (CI/CD) aur